In [1]:
from llama_cpp import Llama
import re
import json
from transformers import pipeline
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import flair
from flair.data import Sentence

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
llm = Llama(model_path="/Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf",  
n_ctx=2048,
n_gpu_layers=-1,
n_batch=512,
callback_manager=callback_manager,
verbose=True,)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dime

In [3]:
# Initialize the pipeline and tokenizer once
triplet_extractor_rebel = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

def extract_text_triplets_rebel(input_text):
    """
    Extracts triplets from the given text.

    Parameters:
    input_text (str): The text from which to extract triplets.

    Returns:
    list: A list of dictionaries, each representing a triplet with 'head', 'type', and 'tail'.
    """
    # Use the tokenizer manually since we need special tokens
    extracted_text = triplet_extractor_rebel.tokenizer.batch_decode([
        triplet_extractor_rebel(input_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]
    ])

    # Function to parse the generated text and extract the triplets
    def extract_triplets(text):
        triplets = []
        relation, subject, object_ = '', '', ''
        text = text.strip()
        current = 'x'
        for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
            if token == "<triplet>":
                current = 't'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                    relation = ''
                subject = ''
            elif token == "<subj>":
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                object_ = ''
            elif token == "<obj>":
                current = 'o'
                relation = ''
            else:
                if current == 't':
                    subject += ' ' + token
                elif current == 's':
                    object_ += ' ' + token
                elif current == 'o':
                    relation += ' ' + token
        if subject != '' and relation != '' and object_ != '':
            triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
        return triplets

    extracted_triplets = extract_triplets(extracted_text[0])
    return extracted_triplets

/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
triplet_extractor_knowgl = pipeline("text2text-generation", model="ibm/knowgl-large")

def extract_text_triplets_knowgl(input_text):
    """
    Extracts triplets from the given text.

    Parameters:
    input_text (str): The text from which to extract triplets.

    Returns:
    list: A list of dictionaries, each representing a triplet with 'head', 'type', and 'tail'.
    """
    extracted_text = triplet_extractor_knowgl.tokenizer.batch_decode([
        triplet_extractor_knowgl(input_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]
    ])
    return extract_triples(extracted_text[0])

def split_spo(input_string):
    pattern = r"\[(.*?)\|\s*(.*?)\s*\|\s*(.*?)\]"
    matches = re.findall(pattern, input_string)
    if matches:
        return matches[0]

def clean_entity(entity_string):
    return entity_string.split("#")[0][1:]

def convert_format(input_text):
    subj, rel, obj = split_spo(input_text)
    return {'head': clean_entity(subj).strip(), 'type': rel,'tail': clean_entity(obj).strip()}

def extract_triples(gen_text):
    gen_text = gen_text.replace("<s><s>", "").replace("</s>", "")
    triples = []
    for triple_text in gen_text.split("$"):
        triples.append(convert_format(triple_text))
    return triples

In [5]:
def extract_text_triplets(input_text):
    """
    Extracts triplets from the given text using both Rebel and KnowGL models,
    and ranks the outputs based on some criteria.

    Parameters:
    input_text (str): The text from which to extract triplets.

    Returns:
    list: A list of dictionaries, each representing a triplet with 'head', 'type', and 'tail',
    sorted based on the ranking criteria.
    """
    # Extract triplets using Rebel model
    triplets_rebel = extract_text_triplets_rebel(input_text)
    
    # Extract triplets using KnowGL model
    triplets_knowgl = extract_text_triplets_knowgl(input_text)
    
    # Combine the triplets from both models
    all_triplets = triplets_rebel + triplets_knowgl
    
    # Rank the triplets based on some criteria (e.g., uniqueness, confidence, etc.)
    # Here, let's assume the ranking criteria is based on the length of the triplets
    unique_triplets = []
    for triplet in all_triplets:
        if triplet not in unique_triplets:
            unique_triplets.append(triplet)
    
    # Rank the triplets based on some criteria (e.g., uniqueness, confidence, etc.)
    # Here, let's assume the ranking criteria is based on the length of the triplets
    ranked_triplets = sorted(unique_triplets, key=lambda x: len(x['type']), reverse=True)
    
    return ranked_triplets

# Example usage:



In [12]:
input_text = "Fado does not work at IIT"
extracted_triplets = extract_text_triplets(input_text)
print(extracted_triplets)

[{'head': 'Fado', 'type': 'educated at', 'tail': 'IIT'}]


In [17]:
# Load the Flair NER model
flair_ner = flair.models.SequenceTagger.load('ner')

def extract_entities(text):
    # Create a Flair Sentence
    sentence = Sentence(text)

    # Run NER on the sentence
    flair_ner.predict(sentence)

    # Initialize lists to store head and tail entities
    head_entities = []
    tail_entities = []

    # Extract entities and their types
    for entity in sentence.get_spans('ner'):
        if entity.tag == 'PER':
            head_entities.append(entity.text)
        else:
            tail_entities.append(entity.text)

    # Create a list of dictionaries containing head and tail entities
    result = [{"head": head, "tail": tail} for head, tail in zip(head_entities, tail_entities)]

    return result

2024-03-15 02:25:30,936 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [23]:
prompt_template = '''<s>[INST] <<SYS>>
Assistant is an expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

The available actions to Assistant are:

- "extract_text_triplets": Useful when Assistant is asked to extract triplets from a given text.
  - To use the extract_triplets tool, Assistant should respond like so:
    {{"action": "extract_text_triplets", "action_input": "Your text here"}}

- "extract_entities": Useful when Assistant is asked to extract entities from a given text.
  - To use the extract_triplets tool, Assistant should respond like so:
    {{"action": "extract_entities", "action_input": "Your text here"}} 

Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: I'm good thanks, how are you?
User: Can you extract all the triplets from this text: "Gràcia is a district of the city of Barcelona, Spain."
Assistant: {{"action": "extract_text_triplets", "action_input": "Gràcia is a district of the city of Barcelona, Spain."}}
User: Also give triples for "obama was US president"
Assistant: {{"action": "extract_text_triplets", "action_input": "obama was US president"}}
User: Can you extract all the entities from this text: "Gràcia is a district of the city of Barcelona, Spain."
Assistant: {{"action": "extract_entities", "action_input": "Gràcia is a district of the city of Barcelona, Spain."}}
User: Also give entities for "obama was US president"
Assistant: {{"action": "extract_entities", "action_input": "obama was US president"}}


<</SYS>>

{0}[/INST]'''

In [19]:
def process_command(command):
    # Put user command into prompt
    prompt = prompt_template.format("User: " + command)
    # Send command to the model
    output = llm(prompt, max_tokens=2000, stop=["User:"])
    response = output['choices'][0]['text']

    # try to find json in the response
    try:
        # Extract json from model response by finding first and last brackets {}
        firstBracketIndex = response.index("{")
        lastBracketIndex = len(response) - response[::-1].index("}")
        jsonString = response[firstBracketIndex:lastBracketIndex]
        responseJson = json.loads(jsonString)
        if responseJson['action'] == 'extract_text_triplets':
            extracted_triplets = extract_text_triplets(responseJson['action_input'])
            return extracted_triplets   
        elif responseJson['action'] == 'extract_entities':
            extracted_entities = extract_entities(responseJson['action_input'])
            return extracted_entities   
    except Exception as e:
        print(e)
    # No json match, just return response
    return response

In [20]:
process_command("Can you please give all the triples for  \"While john F Kennedy and ronald reagan delivered their famous speeches from the safeto of West Berlin, Bruce Springsteen's speaking out against the Berlin Wall in the middle of East Berlin added to the euphoria\". ")


llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      55.59 ms /   169 runs   (    0.33 ms per token,  3040.28 tokens per second)
llama_print_timings: prompt eval time =   12348.61 ms /   519 tokens (   23.79 ms per token,    42.03 tokens per second)
llama_print_timings:        eval time =   13762.48 ms /   168 runs   (   81.92 ms per token,    12.21 tokens per second)
llama_print_timings:       total time =   27075.69 ms /   687 tokens


[{'head': 'Berlin Wall',
  'type': 'located in the administrative territorial entity',
  'tail': 'East Berlin'},
 {'head': 'West Berlin', 'type': 'shares border with', 'tail': 'East Berlin'},
 {'head': 'East Berlin', 'type': 'shares border with', 'tail': 'West Berlin'},
 {'head': 'Berlin Wall', 'type': 'location', 'tail': 'East Berlin'}]

In [24]:
process_command("Can you please give all the entities for  \"While john F Kennedy and ronald reagan delivered their famous speeches from the safeto of West Berlin, Bruce Springsteen's speaking out against the Berlin Wall in the middle of East Berlin added to the euphoria\". ")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      23.79 ms /   107 runs   (    0.22 ms per token,  4497.31 tokens per second)
llama_print_timings: prompt eval time =    2936.05 ms /   109 tokens (   26.94 ms per token,    37.12 tokens per second)
llama_print_timings:        eval time =    9188.82 ms /   106 runs   (   86.69 ms per token,    11.54 tokens per second)
llama_print_timings:       total time =   12617.74 ms /   215 tokens


[{'head': 'john F Kennedy', 'tail': 'West Berlin'},
 {'head': 'Bruce Springsteen', 'tail': 'Berlin Wall'}]

In [25]:
import json

# Function to extract triples for each context in eval.json and create pred.json
def generate_pred_json(eval_file_path, pred_file_path):
    # Load the evaluation data from eval.json
    with open(eval_file_path, 'r') as file:
        eval_data = json.load(file)
    
    # Initialize a list to hold the modified data with extracted triples
    modified_data = []
    
    # Iterate over each item in the evaluation data
    for item in eval_data:
        context = item['context']
        # Prepare the command with the context
        command = f"Can you please give triple for \"{context}\""
        # Use the process_command function to predict the extracted triples
        extracted_triplets = process_command(command)
        # Append the extracted triples to the item under the 'triples' key
        item['triples'] = extracted_triplets
        # Append the modified item to the modified_data list
        modified_data.append(item)
    
    # Write the modified data with extracted triples to pred.json
    with open(pred_file_path, 'w') as file:
        json.dump(modified_data, file, indent=4)

# Example usage


In [28]:
eval_file_path = '/Users/ananyahooda/Desktop/final/data/evaluation_data/conll04_eval.json' # Replace with the actual path to your eval.json file
pred_file_path = '/Users/ananyahooda/Desktop/final/tripleExtractionPythonScript/Final_ensemble_prediction_triples.json' # The output file path
generate_pred_json(eval_file_path, pred_file_path)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      15.62 ms /    51 runs   (    0.31 ms per token,  3265.46 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   11802.93 ms /    51 runs   (  231.43 ms per token,     4.32 tokens per second)
llama_print_timings:       total time =   12046.12 ms /    52 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      23.10 ms /    28 runs   (    0.83 ms per token,  1212.02 tokens per second)
llama_print_timings: prompt eval time =    4942.51 ms /    13 tokens (  380.19 ms per token,     2.63 tokens per second)
llama_print_timings:        eval time =    2135.49 ms /    27 runs   (   79.09 ms per token,    12.64 tokens per second)
llama_print_timings:       to

Extra data: line 3 column 1 (char 202)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =     109.61 ms /   165 runs   (    0.66 ms per token,  1505.41 tokens per second)
llama_print_timings: prompt eval time =     576.98 ms /    36 tokens (   16.03 ms per token,    62.39 tokens per second)
llama_print_timings:        eval time =   14094.08 ms /   164 runs   (   85.94 ms per token,    11.64 tokens per second)
llama_print_timings:       total time =   16111.32 ms /   200 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      20.17 ms /    28 runs   (    0.72 ms per token,  1388.13 tokens per second)
llama_print_timings: prompt eval time =     409.04 ms /    13 tokens (   31.46 ms per token,    31.78 tokens per second)
llama_print_timings:        eval time =    2138.68 ms /    27 runs   (   79.21 ms per token,    12.62 tokens per second)
llama_print_timings:       total time =    2838.11 ms /    40 

Extra data: line 3 column 1 (char 189)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      70.29 ms /    84 runs   (    0.84 ms per token,  1195.03 tokens per second)
llama_print_timings: prompt eval time =     574.02 ms /    35 tokens (   16.40 ms per token,    60.97 tokens per second)
llama_print_timings:        eval time =    6658.20 ms /    83 runs   (   80.22 ms per token,    12.47 tokens per second)
llama_print_timings:       total time =    8111.14 ms /   118 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =     175.22 ms /   223 runs   (    0.79 ms per token,  1272.70 tokens per second)
llama_print_timings: prompt eval time =     694.57 ms /    49 tokens (   14.17 ms per token,    70.55 tokens per second)
llama_print_timings:        eval time =   18283.87 ms /   222 runs   (   82.36 ms per token,    12.14 tokens per second)
llama_print_timings:       total time =   21353.32 ms /   271 

...


llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      19.47 ms /    50 runs   (    0.39 ms per token,  2568.58 tokens per second)
llama_print_timings: prompt eval time =     571.90 ms /    35 tokens (   16.34 ms per token,    61.20 tokens per second)
llama_print_timings:        eval time =    3910.57 ms /    49 runs   (   79.81 ms per token,    12.53 tokens per second)
llama_print_timings:       total time =    4865.83 ms /    84 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      68.60 ms /   154 runs   (    0.45 ms per token,  2244.77 tokens per second)
llama_print_timings: prompt eval time =    2949.71 ms /    38 tokens (   77.62 ms per token,    12.88 tokens per second)
llama_print_timings:        eval time =   12090.18 ms /   153 runs   (   79.02 ms per token,    12.65 tokens per second)
llama_print_timings:       total time =   16392.73 ms /   191 

Extra data: line 3 column 1 (char 231)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      55.70 ms /   153 runs   (    0.36 ms per token,  2747.06 tokens per second)
llama_print_timings: prompt eval time =     596.18 ms /    55 tokens (   10.84 ms per token,    92.25 tokens per second)
llama_print_timings:        eval time =   12696.69 ms /   152 runs   (   83.53 ms per token,    11.97 tokens per second)
llama_print_timings:       total time =   14563.25 ms /   207 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      88.80 ms /   251 runs   (    0.35 ms per token,  2826.45 tokens per second)
llama_print_timings: prompt eval time =    2677.18 ms /    57 tokens (   46.97 ms per token,    21.29 tokens per second)
llama_print_timings:        eval time =   20522.61 ms /   250 runs   (   82.09 ms per token,    12.18 tokens per second)
llama_print_timings:       total time =   25454.28 ms /   307 

Extra data: line 3 column 1 (char 150)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      15.40 ms /    54 runs   (    0.29 ms per token,  3506.72 tokens per second)
llama_print_timings: prompt eval time =     579.84 ms /    38 tokens (   15.26 ms per token,    65.54 tokens per second)
llama_print_timings:        eval time =    4352.87 ms /    53 runs   (   82.13 ms per token,    12.18 tokens per second)
llama_print_timings:       total time =    5232.87 ms /    91 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      13.26 ms /    49 runs   (    0.27 ms per token,  3694.21 tokens per second)
llama_print_timings: prompt eval time =     948.24 ms /    33 tokens (   28.73 ms per token,    34.80 tokens per second)
llama_print_timings:        eval time =    4235.51 ms /    48 runs   (   88.24 ms per token,    11.33 tokens per second)
llama_print_timings:       total time =    5707.70 ms /    81 

In [29]:
# Function to calculate precision, recall, and F1 score
def calculate_scores(tp, total_golden, total_prediction):
    precision = tp / total_prediction if total_prediction > 0 else 0
    recall = tp / total_golden if total_golden > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

# Function to process the files and calculate the scores, considering extras
def evaluate_predictions_corrected(golden_file, prediction_file):
    # Load the golden truths and predictions
    with open(golden_file, 'r') as f:
        golden_data = json.load(f)
    with open(prediction_file, 'r') as f:
        prediction_data = json.load(f)

    tp = 0
    extras = 0

    # Convert golden data and prediction data into dictionaries for easier access
    golden_dict = {item['id']: set(tuple(triple.items()) for triple in item['triples']) for item in golden_data}
    prediction_dict = {item['id']: set(tuple(triple.items()) for triple in item['triples']) for item in prediction_data}

    # Iterate over each instance in the golden data to calculate true positives
    for id, golden_triples in golden_dict.items():
        prediction_triples = prediction_dict.get(id, set())
        tp += len(golden_triples & prediction_triples)

    # Calculate extras in prediction
    for id, prediction_triples in prediction_dict.items():
        if id not in golden_dict:
            extras += len(prediction_triples)
        else:
            unmatched_triples = prediction_triples - golden_dict[id]
            print(unmatched_triples)
            extras += len(unmatched_triples)

    # Calculate micro scores
    total_golden = sum(len(triples) for triples in golden_dict.values())
    total_prediction = sum(len(triples) for triples in prediction_dict.values())
    precision_micro, recall_micro, f1_micro = calculate_scores(tp, total_golden, total_prediction)

    # Calculate macro scores
    total_items = len(golden_dict)
    precision_macro, recall_macro, f1_macro = 0, 0, 0
    for id, golden_triples in golden_dict.items():
        prediction_triples = prediction_dict.get(id, set())
        tp = len(golden_triples & prediction_triples)
        precision, recall, _ = calculate_scores(tp, len(golden_triples), len(prediction_triples))
        precision_macro += precision
        recall_macro += recall
    precision_macro /= total_items
    recall_macro /= total_items
    f1_macro = 2 * (precision_macro * recall_macro) / (precision_macro + recall_macro) if (precision_macro + recall_macro) > 0 else 0

    return {
        'micro': {
            'precision': precision_micro,
            'recall': recall_micro,
            'f1': f1_micro
        },
        'macro': {
            'precision': precision_macro,
            'recall': recall_macro,
            'f1': f1_macro
        },
        'true_positives': tp,
        'extras': extras
    }

## String Removal (The hallucinated responses not in correct format)

In [10]:
golden_file = "/Users/ananyahooda/Desktop/final/golden_truth.json"

In [37]:
import json

# Load the JSON data from the two files
with open(pred_file_path , 'r') as file:
    data1 = json.load(file)

with open(golden_file, 'r') as file:
    data2 = json.load(file)

# Determine the length of both JSON files
length_data1 = len(data1)
length_data2 = len(data2)

# Print the lengths
print(f"Length of the golden truth JSON file: {length_data1}")
print(f"Length of the prediction JSON file: {length_data2}")

# Convert the lists to dictionaries indexed by the 'id' attribute
data1_dict = {item['id']: item for item in data1}
data2_dict = {item['id']: item for item in data2}

# Find the common IDs
common_ids = set(data1_dict.keys()) & set(data2_dict.keys())

# Extract the common data points
common_data1 = [data1_dict[id] for id in common_ids]
common_data2 = [data2_dict[id] for id in common_ids]

# Save the common data points to new JSON files
with open('pred_ensemble_common.json', 'w') as file:
    json.dump(common_data1, file, indent=4)

with open('golden_truth_common.json', 'w') as file: 
    json.dump(common_data2, file, indent=4)

# Print a message to indicate that the new files have been created
print(f"Created new JSON files with common data points: 'common_file1.json' and 'common_file2.json'")

Length of the golden truth JSON file: 288
Length of the prediction JSON file: 262
Created new JSON files with common data points: 'common_file1.json' and 'common_file2.json'


In [38]:
import json

# Load the JSON data from the pred.json file
with open('pred_ensemble_common.json', 'r') as file:
    pred_data = json.load(file)

# Initialize a counter for entries with "triples" as a string
string_triples_count = 0
string_triples = []

# Iterate over the entries and check the type of "triples"
for entry in pred_data:
    if 'triples' in entry and isinstance(entry['triples'], str):
        string_triples_count += 1
        string_triples.append(entry['triples'])

# Print the count of such entries

print(f"Number of entries with 'triples' as a string: {string_triples_count}")

for i, string in enumerate(string_triples, start=1):
    print(f"String {i}: {string}")

Number of entries with 'triples' as a string: 19
String 1:  {"action": "extract_text_triplets", "action_input": "High winds blew on the east slopes of the Rocky Mountains in Montana , with winds gusting to near 50 mph at Livingston ."} 

Here's the expected response format for the extracted triplets or entities:
{"triplets": [["High winds", "blew", "east slopes"], ["winds", "gusted", "near 50 mph"], ["winds", "reached", "Livingston"]], or ["entities": ["High winds", "east slopes", "Rocky Mountains", "Montana", "Livingston"]]} 

Note that the exact format of the output may vary depending on the specific requirements of the task or the NLP tool being used.
String 2:  {"action": "extract_text_triplets", "action_input": "Also on the GOP ballot are insurance broker David Fleischer and former North Miami Mayor John Stembridge.} "

Here are the possible extracted triplets from the given input: [("Also", "on", "the GOP ballot"), ("are", "insurance broker", "David Fleischer"), ("and", "", "), (

In [39]:
import json

# Load the JSON data from the two files
with open('golden_truth_common.json', 'r') as file:
    data1 = json.load(file)

with open('pred_ensemble_common.json', 'r') as file:
    data2 = json.load(file)

# Find the IDs of entries with "triples" as a string in file1
ids_to_remove = [entry['id'] for entry in data2 if 'triples' in entry and isinstance(entry['triples'], str)]

# Remove the entries from both files
filtered_data1 = [entry for entry in data1 if entry['id'] not in ids_to_remove]
filtered_data2 = [entry for entry in data2 if entry['id'] not in ids_to_remove]

# Save the filtered data back to new JSON files
with open('/Users/ananyahooda/Desktop/final/final_comparable_golden_truth.json', 'w') as file:
    json.dump(filtered_data1 , file, indent=4)

with open('/Users/ananyahooda/Desktop/final/comparabe_ensemble_predi.json', 'w') as file:
    json.dump(filtered_data2, file, indent=4)

# Print a message to indicate that the entries have been removed
print(f"Entries with 'triples' as a string have been removed. New files created: 'filtered_file1.json' and 'filtered_file2.json'")

Entries with 'triples' as a string have been removed. New files created: 'filtered_file1.json' and 'filtered_file2.json'


## For mapping 90 relation to 5 relations as in Conll04

In [40]:
mapping =  {'employer': ['derivative work',   'inception', 'instance of', 'owned by', 'owner of', 'part of', 'participant', 
'participant in', 'performer', 'twinned administrative body','occupation', 'field of this occupation', 'member of political party','work location', 'language used', 'participant in', 'participant','owner of', 'owned by', 'member of', 'notable work', 'instance of', 'interested in',  'office held by head of government', 'chief executive officer', 'educated at', 'subclass of','part of', 'office held by head of state','chairperson', 'executive body', 'industry', 'officeholder', 
'position held', 'practiced by', 'language of work or name', 'director / manager', 'employer', 'field of work', 
'language of work or name', 'notable work', 'occupation', 'member of', 'member of political party', 'officeholder',
 'operator', 'position held', 'educated at', 'founded by',
'product or material produced', 'subsidiary', 'work location', 'author', 
'office held by head of government', 'used by', 'uses', 'candidacy in election', 'candidate',  'chairperson', 'head of government' ], 
'headquarters location': ['headquarters location', 'twinned administrative body','applies to jurisdiction', 'legislative body','military branch', 'contains administrative territorial entity', 
'parent organization', 'operating area','legislative body', 'contains administrative territorial entity', 
'headquarters location', 'located in the administrative territorial entity', 'ethnic group', 'language used',
'military branch', 'parent organization', 'applies to jurisdiction'],
 'killed by': ['cause of death', 'perpetrator', 'convicted of', 
'killed by', 'place of death',  'facet of','date of death',  'main subject', 'place of death', 'facet of', 'significant event'], 'location': ['location',  'capital','continent',  'located in time zone', 'shares border with', 
'mountain range', 'located in or next to body of water', 'candidate',   'significant place',   'spouse', 'place of publication',  'target','country', 'located in or next to body of water', 'location', 
 'mouth of the watercourse', 'point in time', 'capital', 'capital of', 
 'shares border with', 'tributary', 'diplomatic relation', 'place of publication', 'spouse'], 
 'residence': [ 'place of birth', 'based on' ,  
 'country of citizenship', 'date of birth' , 'has part', 
 'number of participants', 'history of topic',  'place of birth', 
'country of origin', 'has quality',  
'significant event','occupant', 'relative', 'residence']}

In [44]:
unique_values = set(tuple(value) for value in mapping.values())

# Calculate total length of unique list values (now as tuples)
total_length = sum(len(value) for value in unique_values)

# Count of unique list values
unique_count = len(unique_values)

print("Total length of unique list values:", total_length)
all_values = [item for sublist in mapping.values() for item in sublist]

# Convert the flattened list to a set to remove duplicates
unique_values = set(all_values)

# Count of unique elements across all lists
unique_count = len(unique_values)

print("Count of unique elements across all lists:", unique_count)

Total length of unique list values: 126
Count of unique elements across all lists: 90


In [45]:
with open('/Users/ananyahooda/Desktop/final/comparabe_ensemble_predi.json' , 'r') as file:
    data = json.load(file)

new_data = []
for obj in data:
    new_obj = {
        "title": obj["title"],
        "context": obj["context"],
        "id": obj["id"],
        "triples": []
    }
    for triple in obj["triples"]:
        for key, value in mapping.items():
            if triple["type"] in value:
                triple["type"] = key
                break
        new_obj["triples"].append(triple)
    new_data.append(new_obj)

# Write new JSON objects to a new file
with open("/Users/ananyahooda/Desktop/final/final_ensembled_5rel_nostring.json", "w") as outfile:
    json.dump(new_data, outfile, indent=4)

print("New file 'new_data.json' created with updated types.")

New file 'new_data.json' created with updated types.


In [46]:
scores = evaluate_predictions_corrected('/Users/ananyahooda/Desktop/final/final_comparable_golden_truth.json',"/Users/ananyahooda/Desktop/final/final_ensembled_5rel_nostring.json")
print("compare rebel_rel5 with test_triples")
print("Micro Scores:", scores['micro'])
print("Macro Scores:", scores['macro'])
print(scores['extras'])

{(('head', 'William R. Higgins'), ('type', 'headquarters location'), ('tail', 'U.S. Marine')), (('head', 'William R. Higgins'), ('type', 'employer'), ('tail', 'Organization of the Oppressed on Earth'))}
{(('head', 'King was killed'), ('type', 'headquarters location'), ('tail', 'Memphis, Tenn.')), (('head', 'King'), ('type', 'killed by'), ('tail', 'Memphis, Tenn.')), (('head', 'King was killed'), ('type', 'location'), ('tail', 'April 4, 1968'))}
{(('head', 'shot Sen. Robert F. Kennedy'), ('type', 'killed by'), ('tail', 'Sirhan B. Sirhan')), (('head', 'Sirhan B. Sirhan'), ('type', 'killed by'), ('tail', '22 years ago'))}
{(('head', 'Berlin'), ('type', 'location'), ('tail', 'Federal Republic')), (('head', 'Federal Administrative Court'), ('type', 'location'), ('tail', 'Federal Republic')), (('head', 'Federal Republic'), ('type', 'location'), ('tail', 'Berlin'))}
{(('head', 'King'), ('type', 'killed by'), ('tail', 'Memphis, Tenn.')), (('head', 'killing of King'), ('type', 'location'), ('ta

### NER 

In [47]:
import json

# Function to extract triples for each context in eval.json and create pred.json
def generate_pred_json(eval_file_path, pred_file_path):
    # Load the evaluation data from eval.json
    with open(eval_file_path, 'r') as file:
        eval_data = json.load(file)
    
    # Initialize a list to hold the modified data with extracted triples
    modified_data = []
    
    # Iterate over each item in the evaluation data
    for item in eval_data:
        context = item['context']
        # Prepare the command with the context
        command = f"Can you please give entities for \"{context}\""
        # Use the process_command function to predict the extracted triples
        extracted_triplets = process_command(command)
        # Append the extracted triples to the item under the 'triples' key
        item['triples'] = extracted_triplets
        # Append the modified item to the modified_data list
        modified_data.append(item)
    
    # Write the modified data with extracted triples to pred.json
    with open(pred_file_path, 'w') as file:
        json.dump(modified_data, file, indent=4)

In [48]:
eval_file_path = '/Users/ananyahooda/Desktop/final/data/evaluation_data/conll04_eval.json' # Replace with the actual path to your eval.json file
pred_file_path = '/Users/ananyahooda/Desktop/final/data/Final_entities.json' # The output file path
generate_pred_json(eval_file_path, pred_file_path)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      13.30 ms /    47 runs   (    0.28 ms per token,  3533.57 tokens per second)
llama_print_timings: prompt eval time =    8577.22 ms /    38 tokens (  225.72 ms per token,     4.43 tokens per second)
llama_print_timings:        eval time =    3761.78 ms /    46 runs   (   81.78 ms per token,    12.23 tokens per second)
llama_print_timings:       total time =   12725.52 ms /    84 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =       5.85 ms /    24 runs   (    0.24 ms per token,  4104.67 tokens per second)
llama_print_timings: prompt eval time =     404.45 ms /    13 tokens (   31.11 ms per token,    32.14 tokens per second)
llama_print_timings:        eval time =    1889.79 ms /    23 runs   (   82.16 ms per token,    12.17 tokens per second)
llama_print_timings:       to

Extra data: line 3 column 1 (char 143)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =       6.69 ms /    24 runs   (    0.28 ms per token,  3587.44 tokens per second)
llama_print_timings: prompt eval time =     310.62 ms /    13 tokens (   23.89 ms per token,    41.85 tokens per second)
llama_print_timings:        eval time =    1882.11 ms /    23 runs   (   81.83 ms per token,    12.22 tokens per second)
llama_print_timings:       total time =    2321.05 ms /    36 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      32.43 ms /   120 runs   (    0.27 ms per token,  3700.28 tokens per second)
llama_print_timings: prompt eval time =     585.37 ms /    39 tokens (   15.01 ms per token,    66.62 tokens per second)
llama_print_timings:        eval time =    9824.86 ms /   119 runs   (   82.56 ms per token,    12.11 tokens per second)
llama_print_timings:       total time =   11023.86 ms /   158 

Invalid \escape: line 1 column 171 (char 170)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      76.67 ms /    96 runs   (    0.80 ms per token,  1252.04 tokens per second)
llama_print_timings: prompt eval time =     581.17 ms /    45 tokens (   12.91 ms per token,    77.43 tokens per second)
llama_print_timings:        eval time =    7560.82 ms /    95 runs   (   79.59 ms per token,    12.56 tokens per second)
llama_print_timings:       total time =    9202.96 ms /   140 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      57.01 ms /    76 runs   (    0.75 ms per token,  1333.01 tokens per second)
llama_print_timings: prompt eval time =     636.36 ms /    46 tokens (   13.83 ms per token,    72.29 tokens per second)
llama_print_timings:        eval time =    6118.08 ms /    75 runs   (   81.57 ms per token,    12.26 tokens per second)
llama_print_timings:       total time =    7565.31 ms /   121 

...


llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      51.02 ms /    69 runs   (    0.74 ms per token,  1352.36 tokens per second)
llama_print_timings: prompt eval time =     313.28 ms /    24 tokens (   13.05 ms per token,    76.61 tokens per second)
llama_print_timings:        eval time =    5546.82 ms /    68 runs   (   81.57 ms per token,    12.26 tokens per second)
llama_print_timings:       total time =    6617.59 ms /    92 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      50.49 ms /    90 runs   (    0.56 ms per token,  1782.43 tokens per second)
llama_print_timings: prompt eval time =     589.72 ms /    46 tokens (   12.82 ms per token,    78.00 tokens per second)
llama_print_timings:        eval time =    7556.60 ms /    89 runs   (   84.91 ms per token,    11.78 tokens per second)
llama_print_timings:       total time =    8953.44 ms /   135 

Extra data: line 3 column 1 (char 187)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      16.57 ms /    33 runs   (    0.50 ms per token,  1992.03 tokens per second)
llama_print_timings: prompt eval time =     331.18 ms /    21 tokens (   15.77 ms per token,    63.41 tokens per second)
llama_print_timings:        eval time =    2953.70 ms /    32 runs   (   92.30 ms per token,    10.83 tokens per second)
llama_print_timings:       total time =    3616.12 ms /    53 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      31.78 ms /    48 runs   (    0.66 ms per token,  1510.24 tokens per second)
llama_print_timings: prompt eval time =     579.21 ms /    36 tokens (   16.09 ms per token,    62.15 tokens per second)
llama_print_timings:        eval time =    3802.38 ms /    47 runs   (   80.90 ms per token,    12.36 tokens per second)
llama_print_timings:       total time =    4903.46 ms /    83 

Expecting ',' delimiter: line 1 column 51 (char 50)



llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      28.80 ms /    46 runs   (    0.63 ms per token,  1597.00 tokens per second)
llama_print_timings: prompt eval time =     574.77 ms /    35 tokens (   16.42 ms per token,    60.89 tokens per second)
llama_print_timings:        eval time =    3692.39 ms /    45 runs   (   82.05 ms per token,    12.19 tokens per second)
llama_print_timings:       total time =    4735.73 ms /    80 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   12045.51 ms
llama_print_timings:      sample time =      58.87 ms /   101 runs   (    0.58 ms per token,  1715.79 tokens per second)
llama_print_timings: prompt eval time =     582.56 ms /    38 tokens (   15.33 ms per token,    65.23 tokens per second)
llama_print_timings:        eval time =    8739.17 ms /   100 runs   (   87.39 ms per token,    11.44 tokens per second)
llama_print_timings:       total time =   10310.23 ms /   138 

In [8]:
pred_file_path = '/Users/ananyahooda/Desktop/final/data/Final_entities.json' 

In [16]:
import json

# Load the JSON data from the two files
with open(pred_file_path , 'r') as file:
    data1 = json.load(file)

with open(golden_file, 'r') as file:
    data2 = json.load(file)

# Determine the length of both JSON files
length_data1 = len(data1)
length_data2 = len(data2)

# Print the lengths
print(f"Length of the golden truth JSON file: {length_data1}")
print(f"Length of the prediction JSON file: {length_data2}")

# Convert the lists to dictionaries indexed by the 'id' attribute
data1_dict = {item['id']: item for item in data1}
data2_dict = {item['id']: item for item in data2}

# Find the common IDs
common_ids = set(data1_dict.keys()) & set(data2_dict.keys())

# Extract the common data points
common_data1 = [data1_dict[id] for id in common_ids]
common_data2 = [data2_dict[id] for id in common_ids]

# Save the common data points to new JSON files
with open('/Users/ananyahooda/Desktop/final/pred_common_entities.json', 'w') as file:
    json.dump(common_data1, file, indent=4)

with open('golden_truth_common.json', 'w') as file: 
    json.dump(common_data2, file, indent=4)

# Print a message to indicate that the new files have been created
print(f"Created new JSON files with common data points: 'common_file1.json' and 'common_file2.json'")

Length of the golden truth JSON file: 288
Length of the prediction JSON file: 262
Created new JSON files with common data points: 'common_file1.json' and 'common_file2.json'


In [18]:
# Load the JSON data from the two files
with open(golden_file, 'r') as file:
    data1 = json.load(file)  # Data from the golden truth file

with open(pred_file_path, 'r') as file:
    data2 = json.load(file)  # Data from the prediction file

# Convert the lists to dictionaries indexed by the 'id' attribute
data1_dict = {item['id']: item for item in data1}
data2_dict = {item['id']: item for item in data2}

# Find the IDs that are in the golden truth but not in the prediction
missing_ids = set(data1_dict.keys()) - set(data2_dict.keys())
print(missing_ids)

# Extract the missing data points
missing_data = [data1_dict[id] for id in missing_ids]

# Print the missing data points
print(f"Missing JSON objects from the prediction file:")
for obj in missing_data:
    print(json.dumps(obj, indent=4))  # Using json.dumps for pretty printing

# Optionally, save the missing data points to a new JSON file
with open('missing_in_prediction.json', 'w') as file:
    json.dump(missing_data, file, indent=4)

# Print a message to indicate that the new file with missing data points has been created
print(f"Created new JSON file 'missing_in_prediction.json' with the missing data points.")

set()
Missing JSON objects from the prediction file:
Created new JSON file 'missing_in_prediction.json' with the missing data points.


In [52]:
import json

# Load the JSON data from the pred.json file
with open('pred_common_entities.json', 'r') as file:
    pred_data = json.load(file)

# Initialize a counter for entries with "triples" as a string
string_triples_count = 0
string_triples = []

# Iterate over the entries and check the type of "triples"
for entry in pred_data:
    if 'triples' in entry and isinstance(entry['triples'], str):
        string_triples_count += 1
        string_triples.append(entry['triples'])

# Print the count of such entries

print(f"Number of entries with 'triples' as a string: {string_triples_count}")

for i, string in enumerate(string_triples, start=1):
    print(f"String {i}: {string}")

Number of entries with 'triples' as a string: 13
String 1:  {"action": "extract_entities", "action_input": "The Taiwan information center provides background and research material on China at a suite of offices in Washington 's National Press Building , where several hundred U.S. and international news organizations also have bureaus.}

Possible entities extracted from the text:

- The Taiwan information center
- China
- Washington
- National Press Building
- Several hundred U.S. and international news organizations
String 2:  {"action": "extract_entities", "action_input": "One was Idaho Gov. Cecil Andrus , who said he needed to deal with forest fires that have become % totally out of control."} 

Here's the expected output:
[{"entity": "One"}, {"entity": "Idaho Gov."}, {"entity": "Cecil Andrus"}] 

Note: The actual output may vary depending on the specific named entities recognized in the text.
String 3:  {"action": "extract_entities", "action_input": "The dead woman was identified as

In [51]:
import json

# Load the JSON data from the two files
with open('golden_truth_common.json', 'r') as file:
    data1 = json.load(file)

with open('pred_common_entities.json', 'r') as file:
    data2 = json.load(file)

# Find the IDs of entries with "triples" as a string in file1
ids_to_remove = [entry['id'] for entry in data2 if 'triples' in entry and isinstance(entry['triples'], str)]

# Remove the entries from both files
filtered_data1 = [entry for entry in data1 if entry['id'] not in ids_to_remove]
filtered_data2 = [entry for entry in data2 if entry['id'] not in ids_to_remove]

# Save the filtered data back to new JSON files
with open('/Users/ananyahooda/Desktop/final/final_final_ent_truth.json', 'w') as file:
    json.dump(filtered_data1 , file, indent=4)

with open('/Users/ananyahooda/Desktop/final/final_com_ent.json', 'w') as file:
    json.dump(filtered_data2, file, indent=4)

# Print a message to indicate that the entries have been removed
print(f"Entries with 'triples' as a string have been removed. New files created: 'filtered_file1.json' and 'filtered_file2.json'")

Entries with 'triples' as a string have been removed. New files created: 'filtered_file1.json' and 'filtered_file2.json'


In [55]:
import json

def calculate_metrics(predicted, golden):
    true_positives = len(set(predicted) & set(golden))
    false_positives = len(set(predicted) - set(golden))
    false_negatives = len(set(golden) - set(predicted))

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1

# Load the JSON files
with open('/Users/ananyahooda/Desktop/final/final_com_ent.json', 'r') as f:
    predicted_data = json.load(f)

with open('/Users/ananyahooda/Desktop/final/final_final_ent_truth.json', 'r') as f:
    golden_data = json.load(f)

# Assuming that the JSON files contain lists of dictionaries with 'triples' and 'matched_entities'
predicted_entities = [entity['head'] for item in predicted_data for entity in item['triples']] + \
                  [entity['tail'] for item in predicted_data for entity in item['triples']]
golden_entities = [entity['head'] for item in golden_data for entity in item['triples']] + \
                  [entity['tail'] for item in golden_data for entity in item['triples']]

# Calculate the metrics
precision, recall, f1 = calculate_metrics(predicted_entities, golden_entities)

# Output the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.5399
Recall: 0.3745
F1 Score: 0.4422
